In [157]:
lsh_dataset = pd.DataFrame()
lsh_dataset['tags'] = cleaned_recipes['tags']
lsh_dataset['name'] = cleaned_recipes['name']
lsh_dataset['id'] = cleaned_recipes['id']
lsh_dataset.head()

,tags,name,id
0,"['60-minutes-or-less', 'time-to-make', 'course...",arriba baked winter squash mexican style,137739
1,"['30-minutes-or-less', 'time-to-make', 'course...",a bit different breakfast pizza,31490
2,"['time-to-make', 'course', 'preparation', 'mai...",all in the kitchen chili,112140
3,"['60-minutes-or-less', 'time-to-make', 'course...",alouette potatoes,59389
4,"['weeknight', 'time-to-make', 'course', 'main-...",amish tomato ketchup for canning,44061


In [2]:
 pip install sklearn

Processing c:\users\lamtr\appdata\local\pip\cache\wheels\23\9d\42\5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c\sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-0.22.2.post1-cp36-cp36m-win_amd64.whl (6.5 MB)
  Using cached joblib-0.14.1-py2.py3-none-any.whl (294 kB)
Note: you may need to restart the kernel to use updated packages.


In [158]:
recipes_map_1 = {}  # store wine names and description there 
recipes_min_hash_map_1 = {}  # store every computed minHash for later use
threshold = 0.7
num_perm = 128  # number of permutation

# creating LSH object
lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)

In [159]:
for row in lsh_dataset.itertuples():
    title = row.id
    tmp = re.sub('\|\?|\[|\!|\/|\;|\:|]|\'| ', '', row.tags) #from object retrived from the column we filter it
    desc = tmp.split(',')                                  #we split it and we retrieve a list of string
        
    if title in recipes_map_1:
        continue  # if it exists => skip it

    recipes_map_1[title] = desc

    min_hash = MinHash(num_perm=num_perm)

    for word in desc:
        min_hash.update(word.encode('utf-8'))
        

    lsh.insert(title, min_hash)
    recipes_min_hash_map_1[title] = min_hash

In [160]:
#recipe_map id: tags
for i in range(5):
    key_list = list(recipes_map_1)
    print('\033[1m', key_list[i], '\033[0;0m',':', end = ' ')
    print(recipes_map_1[key_list[i]])

 137739  : ['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']
 31490  : ['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'breakfast', 'main-dish', 'pork', 'american', 'oven', 'easy', 'kid-friendly', 'pizza', 'dietary', 'northeastern-united-states', 'meat', 'equipment']
 112140  : ['time-to-make', 'course', 'preparation', 'main-dish', 'chili', 'crock-pot-slow-cooker', 'dietary', 'equipment', '4-hours-or-less']
 59389  : ['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'potatoes', 'vegetables', 'oven', 'easy', 'dinner-party', 'holiday-event', 'easter', 'cheese', 'stove-top', 'dietary', 'christmas', 'new-years', 'thanksgiving', '

In [ ]:
count = 0
lsh_dataset['number of similar recipe'] = 0
for recipe in recipes_map_1:
    result = lsh.query(recipes_min_hash_map_1[recipe])
    
    ind = lsh_dataset.index[lsh_dataset['id']==recipe]
    #print(ind, recipe,len(result))
    if (len(result) - 1) > 0 :
        lsh_dataset.loc[ind[0],'number of similar recipe'] = len(result) -1
        
        
    #display(ind)
    if count < 10:
        if len(result) > 1:
            print("Recipe %s has %d similar item:" % (recipe, len(result) - 1))
            print(result)
            #for item in result:
                #if item != recipe:
                    #print("    %s" % item)
            print("\n---------")
    count += 1


Recipe 137739 has 9 similar item:
[199430, 254407, 255687, 57193, 164038, 137739, 522854, 187889, 359989, 358007]

---------
Recipe 31490 has 105 similar item:
[302595, 392714, 151571, 293908, 381973, 384021, 159252, 8728, 260125, 294946, 294948, 15399, 461356, 298029, 235054, 259116, 60977, 37946, 51778, 327758, 497236, 364629, 499804, 11869, 29284, 39529, 50285, 293494, 98934, 264321, 462980, 160909, 497295, 104079, 296601, 359072, 322215, 146599, 164524, 35501, 182958, 466094, 106677, 240825, 42682, 21697, 229060, 164052, 459482, 31450, 13023, 314608, 59637, 88319, 62721, 31490, 13060, 13573, 39175, 326417, 255252, 368410, 385818, 378159, 125231, 308025, 226109, 431934, 293185, 212294, 181065, 442185, 315724, 22866, 102234, 25947, 217955, 439144, 310640, 494450, 225138, 178555, 512893, 259460, 369546, 308112, 17300, 12196, 294821, 29094, 285095, 223657, 139690, 47027, 13256, 512463, 311252, 97754, 14310, 12774, 275944, 351211, 26093, 410095, 18935, 358392]

---------
Recipe 112140 h

In [ ]:
b1 = np.array([199430, 254407, 255687, 57193, 164038, 137739, 522854, 187889, 359989, 358007])
b2 = np.array([113029, 137739, 48779, 13966, 202772, 202777, 320794, 195486, 216993, 76838, 403499, 97069, 109616, 326193, 337220, 362182, 14549, 201178, 104035, 202724, 264292, 271974, 373867, 151788, 103151, 337904, 187763, 399091, 222586])


In [ ]:
np.isin(b2, b1)

In [ ]:
a1 = np.array([
    '60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient',
    'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes',
    'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian',
    'winter', 'dietary', 'christmas', 'seasonal', 'squash'
])
a2 = np.array([
    '60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient',
    'cuisine', 'preparation', 'occasion', 'north-american', 'pies-and-tarts',
    'desserts', 'fruit', 'american', 'southern-united-states', 'easy',
    'holiday-event', 'chocolate', 'nuts', 'seasonal'
])

In [ ]:
print(len(a1))
print(np.sum(np.isin(a1, a2)))

In [ ]:
LSH_recipes_tags.loc[LSH_recipes_tags.id == 137739, 'tags'].to_list()

In [110]:
LSH_recipes_tags.loc[LSH_recipes_tags.id.isin([199430, 254407, 255687, 57193, 164038, 137739, 522854, 187889, 359989, 358007]), "tags"].to_list()


["['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']",
 "['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'yams-sweet-potatoes', 'north-american', 'bisques-cream-soups', 'soups-stews', 'potatoes', 'vegetables', 'canadian', 'fall', 'winter', 'seasonal', 'squash']",
 "['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'fall', 'winter', 'seasonal', 'squash']",
 "['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'main-dish', 'potatoes', 'vegetables', 'american', 'vegetarian', 'winter', 'pizza', 'dietary', 'seasonal', 'northeastern-united-states', 's

In [107]:
LSH_recipes_tags.loc[LSH_recipes_tags.id == 164038]['tags'].tolist()


["['course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'beverages', 'fruit', 'mexican', 'easy', 'beginner-cook', 'summer', 'vegetarian', 'dietary', 'seasonal', 'tropical-fruit', 'pineapple']"]

In [111]:
lsh_dataset.loc[lsh_dataset.id.isin([113029, 137739, 48779, 13966, 202772, 202777, 320794, 195486, 216993, 76838, 403499, 97069, 109616, 326193, 337220, 362182, 14549, 201178, 104035, 202724, 264292, 271974, 373867, 151788, 103151, 337904, 187763, 399091, 222586]), "tags"].to_list()


["['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']",
 "['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'potatoes', 'vegetables', 'american', 'southern-united-states', 'easy', 'beginner-cook', 'holiday-event', 'kid-friendly', 'winter', 'dietary', 'seasonal']",
 "['time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'low-protein', 'pies-and-tarts', 'desserts', 'fruit', 'american', 'southern-united-states', 'oven', 'fall', 'holiday-event', 'winter', 'pies', 'dietary', 'christmas', 'thanksgiving', 'seasonal', 'low-in-something', 'apples', 'equipment', '4-hours-or-less']",
 "['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient'

************************************************

In [80]:
LSH_recipes_tags = cleaned_recipes.loc[:, ['id', 'name', 'tags']]

In [81]:
LSH_recipes_tags

,id,name,tags
0,137739,arriba baked winter squash mexican style,"['60-minutes-or-less', 'time-to-make', 'course..."
1,31490,a bit different breakfast pizza,"['30-minutes-or-less', 'time-to-make', 'course..."
2,112140,all in the kitchen chili,"['time-to-make', 'course', 'preparation', 'mai..."
3,59389,alouette potatoes,"['60-minutes-or-less', 'time-to-make', 'course..."
4,44061,amish tomato ketchup for canning,"['weeknight', 'time-to-make', 'course', 'main-..."
...,...,...,...
211760,486161,zydeco soup,"['ham', '60-minutes-or-less', 'time-to-make', ..."
211761,493372,zydeco spice mix,"['15-minutes-or-less', 'time-to-make', 'course..."
211762,308080,zydeco ya ya deviled eggs,"['60-minutes-or-less', 'time-to-make', 'course..."
211763,298512,cookies by design cookies on a stick,"['30-minutes-or-less', 'time-to-make', 'course..."


In [82]:
recipe_map = {}  # key as recipe id and values as tags
recipe_min_hash_map = {}  # store every computed minHash for later use
threshold = 0.7  #similarity threshold
num_perm = 128  #number of permutations or hash functions applied
lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)

Scan through each record or observation in **lsh_dataset**, retrieve id and tags, add the corresponding data to **recipe_map =({id: tag}**, **recipe_min_hash_map = {id: minhashed signatures}**, populate **(id, minhashed signatues)** into **lsh function**)

We randomly scan through tags attribute of 20 records to see how text data is stored and formatted in advance of pre-processing text data for minhash and lsh

In [83]:
for i in range(20):
    field_value = LSH_recipes_tags.loc[np.random.randint(0, len(LSH_recipes_tags)), 'tags']    
    print(field_value)
    print('Type:', type(field_value), end = '\n' + '-'*100 + ' \n')

['time-to-make', 'course', 'main-ingredient', 'preparation', 'low-protein', 'healthy', 'salads', 'fruit', 'easy', 'kid-friendly', 'low-fat', 'dietary', 'low-sodium', 'low-cholesterol', 'low-saturated-fat', 'low-calorie', 'healthy-2', 'low-in-something', 'apples', '4-hours-or-less']
Type: <class 'str'>
---------------------------------------------------------------------------------------------------- 
['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'pies-and-tarts', 'tarts', 'desserts', 'fruit', 'easy', 'dinner-party', 'summer', 'vegetarian', 'dietary', 'seasonal', 'berries', 'raspberries', 'strawberries', '3-steps-or-less']
Type: <class 'str'>
---------------------------------------------------------------------------------------------------- 
['15-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'for-large-groups', '5-ingredients-or-less', 'condiments-etc', 'fruit', 'a

We can see that the values of the attribute ***tags*** are stored as text (type: str), including unncessary symbols such as: ',[,], white space, so we only need to retrieve the main contents of each value such as: 'time-to-make', 'course'..., and remove other symbols. Finally, we put them into a list which is used for the minhash and LSH

In [84]:
for row in LSH_recipes_tags.itertuples():
    ids = row.id
    tags = re.sub(r'[\'\"\[\]]', '', row.tags).split(',') #remove unneccessary symbols, 
#     tags = re.sub('\|\?|\[|\!|\/|\;|\:|]|\'', '', row.tags).split(',') #remove unneccessary symbols, 

                                                           #use list to store the tags for each record
    if ids in recipe_map:
        continue 
    recipe_map[ids] = tags
    min_hash = MinHash(num_perm = num_perm)  #assign MinHash function to min_hash with num_perm
    for tag in tags: #
        min_hash.update(tag.encode('utf-8')) # let's populate each tag in tags into minhassh function
    lsh.insert(ids, min_hash) #populate minhash signatures for each id into lsh
    recipe_min_hash_map[ids] = min_hash    

In [85]:
#recipe_map id: tags
for i in range(5):
    key_list = list(recipe_map)
    print('\033[1m', key_list[i], '\033[0;0m',':', end = ' ')
    print(recipe_map[key_list[i]])

 137739  : ['60-minutes-or-less', ' time-to-make', ' course', ' main-ingredient', ' cuisine', ' preparation', ' occasion', ' north-american', ' side-dishes', ' vegetables', ' mexican', ' easy', ' fall', ' holiday-event', ' vegetarian', ' winter', ' dietary', ' christmas', ' seasonal', ' squash']
 31490  : ['30-minutes-or-less', ' time-to-make', ' course', ' main-ingredient', ' cuisine', ' preparation', ' occasion', ' north-american', ' breakfast', ' main-dish', ' pork', ' american', ' oven', ' easy', ' kid-friendly', ' pizza', ' dietary', ' northeastern-united-states', ' meat', ' equipment']
 112140  : ['time-to-make', ' course', ' preparation', ' main-dish', ' chili', ' crock-pot-slow-cooker', ' dietary', ' equipment', ' 4-hours-or-less']
 59389  : ['60-minutes-or-less', ' time-to-make', ' course', ' main-ingredient', ' preparation', ' occasion', ' side-dishes', ' eggs-dairy', ' potatoes', ' vegetables', ' oven', ' easy', ' dinner-party', ' holiday-event', ' easter', ' cheese', ' stov

In [86]:
#recipe_map id: tags
for i in range(5):
    key_list = list(recipe_map)
    print('\033[1m', key_list[i], '\033[0;0m',':', end = ' ')
    print(recipe_map[key_list[i]])

 137739  : ['60-minutes-or-less', ' time-to-make', ' course', ' main-ingredient', ' cuisine', ' preparation', ' occasion', ' north-american', ' side-dishes', ' vegetables', ' mexican', ' easy', ' fall', ' holiday-event', ' vegetarian', ' winter', ' dietary', ' christmas', ' seasonal', ' squash']
 31490  : ['30-minutes-or-less', ' time-to-make', ' course', ' main-ingredient', ' cuisine', ' preparation', ' occasion', ' north-american', ' breakfast', ' main-dish', ' pork', ' american', ' oven', ' easy', ' kid-friendly', ' pizza', ' dietary', ' northeastern-united-states', ' meat', ' equipment']
 112140  : ['time-to-make', ' course', ' preparation', ' main-dish', ' chili', ' crock-pot-slow-cooker', ' dietary', ' equipment', ' 4-hours-or-less']
 59389  : ['60-minutes-or-less', ' time-to-make', ' course', ' main-ingredient', ' preparation', ' occasion', ' side-dishes', ' eggs-dairy', ' potatoes', ' vegetables', ' oven', ' easy', ' dinner-party', ' holiday-event', ' easter', ' cheese', ' stov

In [87]:
#recipe_min_hash_map, id: minhash signature
recipe_min_hash_map

{137739: <datasketch.minhash.MinHash at 0x2325ed7a288>,
 31490: <datasketch.minhash.MinHash at 0x2325edf8408>,
 112140: <datasketch.minhash.MinHash at 0x2325ef3e808>,
 59389: <datasketch.minhash.MinHash at 0x2325ed667c8>,
 44061: <datasketch.minhash.MinHash at 0x2325edae748>,
 25274: <datasketch.minhash.MinHash at 0x2325ed467c8>,
 67888: <datasketch.minhash.MinHash at 0x2325ed31f08>,
 75452: <datasketch.minhash.MinHash at 0x2325ed23fc8>,
 109439: <datasketch.minhash.MinHash at 0x2325ed1a9c8>,
 39959: <datasketch.minhash.MinHash at 0x2325ed131c8>,
 43026: <datasketch.minhash.MinHash at 0x2325ed037c8>,
 23933: <datasketch.minhash.MinHash at 0x2325ecf89c8>,
 8559: <datasketch.minhash.MinHash at 0x2325ecdd208>,
 23850: <datasketch.minhash.MinHash at 0x2325ecc8248>,
 76808: <datasketch.minhash.MinHash at 0x2325ecabfc8>,
 24701: <datasketch.minhash.MinHash at 0x2325ec98288>,
 83873: <datasketch.minhash.MinHash at 0x2325ec92e88>,
 107699: <datasketch.minhash.MinHash at 0x2325ec7ebc8>,
 49262:

Query each recipe in **LSH_recipes_tags** for consideration of their similarity based on similarity threshold and number of hash functions applied

In [88]:
count = 0
for ids in recipe_map:
    result = lsh.query(recipe_min_hash_map[ids]) #check similarty for the given recipe id (ids) agaisnt all ids 
    
    #update the similarities of the given id, len - 1 is to exclude the given id.
    LSH_recipes_tags.loc[LSH_recipes_tags.id == ids, 'num_similarity'] = len(result) - 1 
    
    if count<10:
        if len(result) > 1:
            print('Recipe ID {} has {} similar recipes\n.'.format(ids, len(result) - 1))
            print(result)
            print('---------------------------------------')
            print('---------------------------------------')
            count += 1

Recipe ID 137739 has 38 similar recipes
.
[197642, 137739, 473742, 107023, 193039, 193811, 202772, 202777, 192410, 119971, 76838, 370086, 189352, 267953, 145462, 192568, 143810, 402499, 337220, 359363, 39754, 46283, 397643, 85330, 12117, 338134, 230872, 105048, 261978, 368474, 345054, 101983, 220896, 379235, 266214, 373867, 141291, 359541, 114809]
---------------------------------------
---------------------------------------
Recipe ID 31490 has 48 similar recipes
.
[46336, 31490, 492420, 462980, 46220, 258062, 17300, 211864, 29594, 39835, 136732, 26656, 175905, 15140, 31653, 487077, 164524, 63276, 37678, 79534, 33839, 29490, 15414, 513593, 149949, 28607, 15039, 69441, 44994, 26561, 46276, 99009, 21697, 43091, 368851, 55636, 30299, 171997, 76893, 210528, 34528, 29282, 41573, 92659, 487671, 234745, 180860, 42749, 11262]
---------------------------------------
---------------------------------------
Recipe ID 112140 has 44 similar recipes
.
[164993, 276355, 448521, 105483, 112140, 94221,

KeyboardInterrupt: 

In [157]:
lsh_dataset = pd.DataFrame()
lsh_dataset['tags'] = cleaned_recipes['tags']
lsh_dataset['name'] = cleaned_recipes['name']
lsh_dataset['id'] = cleaned_recipes['id']
lsh_dataset.head()

,tags,name,id
0,"['60-minutes-or-less', 'time-to-make', 'course...",arriba baked winter squash mexican style,137739
1,"['30-minutes-or-less', 'time-to-make', 'course...",a bit different breakfast pizza,31490
2,"['time-to-make', 'course', 'preparation', 'mai...",all in the kitchen chili,112140
3,"['60-minutes-or-less', 'time-to-make', 'course...",alouette potatoes,59389
4,"['weeknight', 'time-to-make', 'course', 'main-...",amish tomato ketchup for canning,44061


In [158]:
recipes_map_1 = {}  # store wine names and description there 
recipes_min_hash_map_1 = {}  # store every computed minHash for later use
threshold = 0.7
num_perm = 128  # number of permutation

# creating LSH object
lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)

In [159]:
for row in lsh_dataset.itertuples():
    title = row.id
    tmp = re.sub('\|\?|\[|\!|\/|\;|\:|]|\'| ', '', row.tags) #from object retrived from the column we filter it
    desc = tmp.split(',')                                  #we split it and we retrieve a list of string
        
    if title in recipes_map_1:
        continue  # if it exists => skip it

    recipes_map_1[title] = desc

    min_hash = MinHash(num_perm=num_perm)

    for word in desc:
        min_hash.update(word.encode('utf-8'))
        

    lsh.insert(title, min_hash)
    recipes_min_hash_map_1[title] = min_hash

In [160]:
#recipe_map id: tags
for i in range(5):
    key_list = list(recipes_map_1)
    print('\033[1m', key_list[i], '\033[0;0m',':', end = ' ')
    print(recipes_map_1[key_list[i]])

 137739  : ['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']
 31490  : ['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'breakfast', 'main-dish', 'pork', 'american', 'oven', 'easy', 'kid-friendly', 'pizza', 'dietary', 'northeastern-united-states', 'meat', 'equipment']
 112140  : ['time-to-make', 'course', 'preparation', 'main-dish', 'chili', 'crock-pot-slow-cooker', 'dietary', 'equipment', '4-hours-or-less']
 59389  : ['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'potatoes', 'vegetables', 'oven', 'easy', 'dinner-party', 'holiday-event', 'easter', 'cheese', 'stove-top', 'dietary', 'christmas', 'new-years', 'thanksgiving', '

In [ ]:
count = 0
lsh_dataset['number of similar recipe'] = 0
for recipe in recipes_map_1:
    result = lsh.query(recipes_min_hash_map_1[recipe])
    
    ind = lsh_dataset.index[lsh_dataset['id']==recipe]
    #print(ind, recipe,len(result))
    if (len(result) - 1) > 0 :
        lsh_dataset.loc[ind[0],'number of similar recipe'] = len(result) -1
        
        
    #display(ind)
    if count < 10:
        if len(result) > 1:
            print("Recipe %s has %d similar item:" % (recipe, len(result) - 1))
            print(result)
            #for item in result:
                #if item != recipe:
                    #print("    %s" % item)
            print("\n---------")
    count += 1


Recipe 137739 has 9 similar item:
[199430, 254407, 255687, 57193, 164038, 137739, 522854, 187889, 359989, 358007]

---------
Recipe 31490 has 105 similar item:
[302595, 392714, 151571, 293908, 381973, 384021, 159252, 8728, 260125, 294946, 294948, 15399, 461356, 298029, 235054, 259116, 60977, 37946, 51778, 327758, 497236, 364629, 499804, 11869, 29284, 39529, 50285, 293494, 98934, 264321, 462980, 160909, 497295, 104079, 296601, 359072, 322215, 146599, 164524, 35501, 182958, 466094, 106677, 240825, 42682, 21697, 229060, 164052, 459482, 31450, 13023, 314608, 59637, 88319, 62721, 31490, 13060, 13573, 39175, 326417, 255252, 368410, 385818, 378159, 125231, 308025, 226109, 431934, 293185, 212294, 181065, 442185, 315724, 22866, 102234, 25947, 217955, 439144, 310640, 494450, 225138, 178555, 512893, 259460, 369546, 308112, 17300, 12196, 294821, 29094, 285095, 223657, 139690, 47027, 13256, 512463, 311252, 97754, 14310, 12774, 275944, 351211, 26093, 410095, 18935, 358392]

---------
Recipe 112140 h

In [ ]:
b1 = np.array([199430, 254407, 255687, 57193, 164038, 137739, 522854, 187889, 359989, 358007])
b2 = np.array([113029, 137739, 48779, 13966, 202772, 202777, 320794, 195486, 216993, 76838, 403499, 97069, 109616, 326193, 337220, 362182, 14549, 201178, 104035, 202724, 264292, 271974, 373867, 151788, 103151, 337904, 187763, 399091, 222586])


In [ ]:
np.isin(b2, b1)

In [ ]:
a1 = np.array([
    '60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient',
    'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes',
    'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian',
    'winter', 'dietary', 'christmas', 'seasonal', 'squash'
])
a2 = np.array([
    '60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient',
    'cuisine', 'preparation', 'occasion', 'north-american', 'pies-and-tarts',
    'desserts', 'fruit', 'american', 'southern-united-states', 'easy',
    'holiday-event', 'chocolate', 'nuts', 'seasonal'
])

In [ ]:
print(len(a1))
print(np.sum(np.isin(a1, a2)))

In [ ]:
LSH_recipes_tags.loc[LSH_recipes_tags.id == 137739, 'tags'].to_list()

In [110]:
LSH_recipes_tags.loc[LSH_recipes_tags.id.isin([199430, 254407, 255687, 57193, 164038, 137739, 522854, 187889, 359989, 358007]), "tags"].to_list()


["['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']",
 "['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'yams-sweet-potatoes', 'north-american', 'bisques-cream-soups', 'soups-stews', 'potatoes', 'vegetables', 'canadian', 'fall', 'winter', 'seasonal', 'squash']",
 "['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'fall', 'winter', 'seasonal', 'squash']",
 "['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'main-dish', 'potatoes', 'vegetables', 'american', 'vegetarian', 'winter', 'pizza', 'dietary', 'seasonal', 'northeastern-united-states', 's

In [107]:
LSH_recipes_tags.loc[LSH_recipes_tags.id == 164038]['tags'].tolist()


["['course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'beverages', 'fruit', 'mexican', 'easy', 'beginner-cook', 'summer', 'vegetarian', 'dietary', 'seasonal', 'tropical-fruit', 'pineapple']"]

In [111]:
lsh_dataset.loc[lsh_dataset.id.isin([113029, 137739, 48779, 13966, 202772, 202777, 320794, 195486, 216993, 76838, 403499, 97069, 109616, 326193, 337220, 362182, 14549, 201178, 104035, 202724, 264292, 271974, 373867, 151788, 103151, 337904, 187763, 399091, 222586]), "tags"].to_list()


["['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']",
 "['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'potatoes', 'vegetables', 'american', 'southern-united-states', 'easy', 'beginner-cook', 'holiday-event', 'kid-friendly', 'winter', 'dietary', 'seasonal']",
 "['time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'low-protein', 'pies-and-tarts', 'desserts', 'fruit', 'american', 'southern-united-states', 'oven', 'fall', 'holiday-event', 'winter', 'pies', 'dietary', 'christmas', 'thanksgiving', 'seasonal', 'low-in-something', 'apples', 'equipment', '4-hours-or-less']",
 "['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient'

************************************************

In [80]:
LSH_recipes_tags = cleaned_recipes.loc[:, ['id', 'name', 'tags']]

In [81]:
LSH_recipes_tags

,id,name,tags
0,137739,arriba baked winter squash mexican style,"['60-minutes-or-less', 'time-to-make', 'course..."
1,31490,a bit different breakfast pizza,"['30-minutes-or-less', 'time-to-make', 'course..."
2,112140,all in the kitchen chili,"['time-to-make', 'course', 'preparation', 'mai..."
3,59389,alouette potatoes,"['60-minutes-or-less', 'time-to-make', 'course..."
4,44061,amish tomato ketchup for canning,"['weeknight', 'time-to-make', 'course', 'main-..."
...,...,...,...
211760,486161,zydeco soup,"['ham', '60-minutes-or-less', 'time-to-make', ..."
211761,493372,zydeco spice mix,"['15-minutes-or-less', 'time-to-make', 'course..."
211762,308080,zydeco ya ya deviled eggs,"['60-minutes-or-less', 'time-to-make', 'course..."
211763,298512,cookies by design cookies on a stick,"['30-minutes-or-less', 'time-to-make', 'course..."


In [82]:
recipe_map = {}  # key as recipe id and values as tags
recipe_min_hash_map = {}  # store every computed minHash for later use
threshold = 0.7  #similarity threshold
num_perm = 128  #number of permutations or hash functions applied
lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)

Scan through each record or observation in **lsh_dataset**, retrieve id and tags, add the corresponding data to **recipe_map =({id: tag}**, **recipe_min_hash_map = {id: minhashed signatures}**, populate **(id, minhashed signatues)** into **lsh function**)

We randomly scan through tags attribute of 20 records to see how text data is stored and formatted in advance of pre-processing text data for minhash and lsh

In [83]:
for i in range(20):
    field_value = LSH_recipes_tags.loc[np.random.randint(0, len(LSH_recipes_tags)), 'tags']    
    print(field_value)
    print('Type:', type(field_value), end = '\n' + '-'*100 + ' \n')

['time-to-make', 'course', 'main-ingredient', 'preparation', 'low-protein', 'healthy', 'salads', 'fruit', 'easy', 'kid-friendly', 'low-fat', 'dietary', 'low-sodium', 'low-cholesterol', 'low-saturated-fat', 'low-calorie', 'healthy-2', 'low-in-something', 'apples', '4-hours-or-less']
Type: <class 'str'>
---------------------------------------------------------------------------------------------------- 
['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'pies-and-tarts', 'tarts', 'desserts', 'fruit', 'easy', 'dinner-party', 'summer', 'vegetarian', 'dietary', 'seasonal', 'berries', 'raspberries', 'strawberries', '3-steps-or-less']
Type: <class 'str'>
---------------------------------------------------------------------------------------------------- 
['15-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'for-large-groups', '5-ingredients-or-less', 'condiments-etc', 'fruit', 'a

We can see that the values of the attribute ***tags*** are stored as text (type: str), including unncessary symbols such as: ',[,], white space, so we only need to retrieve the main contents of each value such as: 'time-to-make', 'course'..., and remove other symbols. Finally, we put them into a list which is used for the minhash and LSH

In [84]:
for row in LSH_recipes_tags.itertuples():
    ids = row.id
    tags = re.sub(r'[\'\"\[\]]', '', row.tags).split(',') #remove unneccessary symbols, 
#     tags = re.sub('\|\?|\[|\!|\/|\;|\:|]|\'', '', row.tags).split(',') #remove unneccessary symbols, 

                                                           #use list to store the tags for each record
    if ids in recipe_map:
        continue 
    recipe_map[ids] = tags
    min_hash = MinHash(num_perm = num_perm)  #assign MinHash function to min_hash with num_perm
    for tag in tags: #
        min_hash.update(tag.encode('utf-8')) # let's populate each tag in tags into minhassh function
    lsh.insert(ids, min_hash) #populate minhash signatures for each id into lsh
    recipe_min_hash_map[ids] = min_hash    

In [85]:
#recipe_map id: tags
for i in range(5):
    key_list = list(recipe_map)
    print('\033[1m', key_list[i], '\033[0;0m',':', end = ' ')
    print(recipe_map[key_list[i]])

 137739  : ['60-minutes-or-less', ' time-to-make', ' course', ' main-ingredient', ' cuisine', ' preparation', ' occasion', ' north-american', ' side-dishes', ' vegetables', ' mexican', ' easy', ' fall', ' holiday-event', ' vegetarian', ' winter', ' dietary', ' christmas', ' seasonal', ' squash']
 31490  : ['30-minutes-or-less', ' time-to-make', ' course', ' main-ingredient', ' cuisine', ' preparation', ' occasion', ' north-american', ' breakfast', ' main-dish', ' pork', ' american', ' oven', ' easy', ' kid-friendly', ' pizza', ' dietary', ' northeastern-united-states', ' meat', ' equipment']
 112140  : ['time-to-make', ' course', ' preparation', ' main-dish', ' chili', ' crock-pot-slow-cooker', ' dietary', ' equipment', ' 4-hours-or-less']
 59389  : ['60-minutes-or-less', ' time-to-make', ' course', ' main-ingredient', ' preparation', ' occasion', ' side-dishes', ' eggs-dairy', ' potatoes', ' vegetables', ' oven', ' easy', ' dinner-party', ' holiday-event', ' easter', ' cheese', ' stov

In [86]:
#recipe_map id: tags
for i in range(5):
    key_list = list(recipe_map)
    print('\033[1m', key_list[i], '\033[0;0m',':', end = ' ')
    print(recipe_map[key_list[i]])

 137739  : ['60-minutes-or-less', ' time-to-make', ' course', ' main-ingredient', ' cuisine', ' preparation', ' occasion', ' north-american', ' side-dishes', ' vegetables', ' mexican', ' easy', ' fall', ' holiday-event', ' vegetarian', ' winter', ' dietary', ' christmas', ' seasonal', ' squash']
 31490  : ['30-minutes-or-less', ' time-to-make', ' course', ' main-ingredient', ' cuisine', ' preparation', ' occasion', ' north-american', ' breakfast', ' main-dish', ' pork', ' american', ' oven', ' easy', ' kid-friendly', ' pizza', ' dietary', ' northeastern-united-states', ' meat', ' equipment']
 112140  : ['time-to-make', ' course', ' preparation', ' main-dish', ' chili', ' crock-pot-slow-cooker', ' dietary', ' equipment', ' 4-hours-or-less']
 59389  : ['60-minutes-or-less', ' time-to-make', ' course', ' main-ingredient', ' preparation', ' occasion', ' side-dishes', ' eggs-dairy', ' potatoes', ' vegetables', ' oven', ' easy', ' dinner-party', ' holiday-event', ' easter', ' cheese', ' stov

In [87]:
#recipe_min_hash_map, id: minhash signature
recipe_min_hash_map

{137739: <datasketch.minhash.MinHash at 0x2325ed7a288>,
 31490: <datasketch.minhash.MinHash at 0x2325edf8408>,
 112140: <datasketch.minhash.MinHash at 0x2325ef3e808>,
 59389: <datasketch.minhash.MinHash at 0x2325ed667c8>,
 44061: <datasketch.minhash.MinHash at 0x2325edae748>,
 25274: <datasketch.minhash.MinHash at 0x2325ed467c8>,
 67888: <datasketch.minhash.MinHash at 0x2325ed31f08>,
 75452: <datasketch.minhash.MinHash at 0x2325ed23fc8>,
 109439: <datasketch.minhash.MinHash at 0x2325ed1a9c8>,
 39959: <datasketch.minhash.MinHash at 0x2325ed131c8>,
 43026: <datasketch.minhash.MinHash at 0x2325ed037c8>,
 23933: <datasketch.minhash.MinHash at 0x2325ecf89c8>,
 8559: <datasketch.minhash.MinHash at 0x2325ecdd208>,
 23850: <datasketch.minhash.MinHash at 0x2325ecc8248>,
 76808: <datasketch.minhash.MinHash at 0x2325ecabfc8>,
 24701: <datasketch.minhash.MinHash at 0x2325ec98288>,
 83873: <datasketch.minhash.MinHash at 0x2325ec92e88>,
 107699: <datasketch.minhash.MinHash at 0x2325ec7ebc8>,
 49262:

Query each recipe in **LSH_recipes_tags** for consideration of their similarity based on similarity threshold and number of hash functions applied

In [88]:
count = 0
for ids in recipe_map:
    result = lsh.query(recipe_min_hash_map[ids]) #check similarty for the given recipe id (ids) agaisnt all ids 
    
    #update the similarities of the given id, len - 1 is to exclude the given id.
    LSH_recipes_tags.loc[LSH_recipes_tags.id == ids, 'num_similarity'] = len(result) - 1 
    
    if count<10:
        if len(result) > 1:
            print('Recipe ID {} has {} similar recipes\n.'.format(ids, len(result) - 1))
            print(result)
            print('---------------------------------------')
            print('---------------------------------------')
            count += 1

Recipe ID 137739 has 38 similar recipes
.
[197642, 137739, 473742, 107023, 193039, 193811, 202772, 202777, 192410, 119971, 76838, 370086, 189352, 267953, 145462, 192568, 143810, 402499, 337220, 359363, 39754, 46283, 397643, 85330, 12117, 338134, 230872, 105048, 261978, 368474, 345054, 101983, 220896, 379235, 266214, 373867, 141291, 359541, 114809]
---------------------------------------
---------------------------------------
Recipe ID 31490 has 48 similar recipes
.
[46336, 31490, 492420, 462980, 46220, 258062, 17300, 211864, 29594, 39835, 136732, 26656, 175905, 15140, 31653, 487077, 164524, 63276, 37678, 79534, 33839, 29490, 15414, 513593, 149949, 28607, 15039, 69441, 44994, 26561, 46276, 99009, 21697, 43091, 368851, 55636, 30299, 171997, 76893, 210528, 34528, 29282, 41573, 92659, 487671, 234745, 180860, 42749, 11262]
---------------------------------------
---------------------------------------
Recipe ID 112140 has 44 similar recipes
.
[164993, 276355, 448521, 105483, 112140, 94221,

KeyboardInterrupt: 

In [ ]:
for i in range(len(a)):
    if i + 4 > len(a):
        break
    